# Проект по SQL

## Описание проекта

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Таблица **`books`**. Данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Таблица **`authors`**. Данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

Таблица **`publishers`**. Данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

Таблица **`ratings`**. Данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Таблица **`reviews`**. Данные о пользовательских обзорах на книги:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Ознакомление с данными

In [1]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa

In [2]:
# устанавливаем параметры
db_config = {
            'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db' # название базы данных
            }

connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

In [3]:
# список таблиц
table_list = ['books', 'authors', 'publishers', 'ratings', 'reviews']

In [4]:
# Выведем первые строки
for table in table_list:
    print(f'Таблица {table}')
    
    # выводим первые 5 строк таблицы table
    query = f'''SELECT * FROM {table}'''
    display(get_sql_data(query).head())
    print(f'Количество записей: {get_sql_data(query).shape[0]} \n')
    print('')

Таблица books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Количество записей: 1000 


Таблица authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Количество записей: 636 


Таблица publishers


...

Количество записей: 340 


Таблица ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Количество записей: 6456 


Таблица reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Количество записей: 2793 




## Задания

**Задача 1**. Посчитаем, сколько книг вышло после 1 января 2000 года

In [5]:
# формируем запрос и выводим данные
query = '''
            SELECT COUNT(book_id)
            FROM books
            WHERE publication_date > '2000-01-01';
        '''

get_sql_data(query)

,count
0,819


После 1 января 2000 года было выпущено 819 книг.

**Задача 2**. Для каждой книги посчитаем количество обзоров и среднюю оценку.

In [10]:
# формируем запрос и выводим данные
query = '''
            SELECT b.title, rev.cnt_reviews, rat.avg_rating
            FROM books b
            LEFT JOIN (SELECT book_id, COUNT(review_id) AS cnt_reviews
                       FROM reviews
                       GROUP BY book_id) rev ON b.book_id = rev.book_id
            LEFT JOIN (SELECT DISTINCT book_id, AVG(rating) OVER(PARTITION BY book_id) AS avg_rating
                       FROM ratings) rat ON rat.book_id = b.book_id
            ORDER BY rat.avg_rating DESC
          
          ;
        '''

get_sql_data(query)

,title,cnt_reviews,avg_rating
0,Misty of Chincoteague (Misty #1),2.0,5.00
1,My Name Is Asher Lev,2.0,5.00
2,Arrows of the Queen (Heralds of Valdemar #1),2.0,5.00
3,Piercing the Darkness (Darkness #2),2.0,5.00
4,Stone of Farewell (Memory Sorrow and Thorn #2),1.0,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3.0,2.25
996,His Excellency: George Washington,2.0,2.00
997,Junky,2.0,2.00
998,Drowning Ruth,3.0,2.00


Книги с высоким средним рейтингом не обязательно должны иметь большое количество обзоров.

**Задача 3**. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [7]:
# формируем запрос и выводим данные
query = '''
            SELECT publisher
            FROM publishers
            WHERE publisher_id IN (SELECT publisher_id
                                   FROM books
                                   WHERE num_pages > 50
                                   GROUP BY publisher_id
                                   ORDER BY COUNT(book_id) DESC
                                   LIMIT 1);
        '''

get_sql_data(query)

,publisher
0,Penguin Books


Penguin Books выпустили наибольшее количество книг толще 50 страниц.

**Задача 4**. Определим автора с самой высокой средней оценкой книг. Будем учитывать только книги с 50 и более оценками.

In [8]:
# формируем запрос и выводим данные
query = '''
            WITH
            top_avg_rating AS (SELECT distinct b.author_id, AVG(rat.rating) OVER(PARTITION BY b.author_id) AS avg_rating
                               FROM ratings rat
                               LEFT JOIN books b ON rat.book_id = b.book_id
                               WHERE b.book_id IN (SELECT book_id
                                                   FROM ratings
                                                   GROUP BY book_id
                                                   HAVING count(rating_id) >= 50)
                               ORDER BY avg_rating DESC)
           
            SELECT top.author_id, top.avg_rating, a.author
            FROM top_avg_rating top
            LEFT JOIN authors a ON a.author_id = top.author_id
            LIMIT 1
        '''

get_sql_data(query)

,author_id,avg_rating,author
0,236,4.287097,J.K. Rowling/Mary GrandPré


Автор с самой высокой средней оценкой книг 4.29 является J.K. Rowling/Mary GrandPré.

**Задача 5**. Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [9]:
# формируем запрос и выводим данные
query = '''
        WITH
        u AS (SELECT username, COUNT(review_id) AS cnt_review
              FROM reviews rev
              WHERE username IN (SELECT username
                                 FROM ratings
                                 GROUP BY username
                                 HAVING COUNT(rating_id) > 48)
              GROUP BY username)
        
        SELECT AVG(cnt_review) AS avg_review
        FROM u
        '''

get_sql_data(query)

,avg_review
0,24.0


Среднее количество обзоров пользователей, которые поставили более 48 оценок, составляет 24.

## Выводы

1. После 1 января 2000 года было выпущено 819 книг;
2. Книги с высоким средним рейтингом не обязательно должны иметь большое количество обзоров;
3. Penguin Books выпустили наибольшее количество книг толще 50 страниц;
4. Автор с самой высокой средней оценкой книг 4.29 является J.K. Rowling/Mary GrandPré;
5. Среднее количество обзоров пользователей, которые поставили более 48 оценок, составляет 24.